In [1]:
from utils_cellpose import list_images
from pathlib import Path

In [2]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("../raw_data/test_data")

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

nuclei_preds_path =  directory_path / "nuclei_preds"

In [3]:
import tifffile

roi_name = "full_image"

filename = Path(images[0]).stem

# Read the nuclei predictions per ROI
nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")


In [4]:
import dask.array as da
import numpy as np
import pyclesperanto_prototype as cle

def simulate_cytoplasm_dask(nuclei_labels, dilation_radius=2, erosion_radius=0, chunk_size=(512, 512)):
    """
    Simulate the cytoplasm from nuclei labels using Dask for large datasets.

    Args:
        nuclei_labels (ndarray or dask.array): Input nuclei labels array.
        dilation_radius (int): Radius for dilating nuclei.
        erosion_radius (int): Radius for eroding nuclei.
        chunk_size (tuple): Chunk size for Dask array.

    Returns:
        dask.array: Cytoplasm simulation as a Dask array.
    """
    # Convert nuclei_labels to a Dask array if not already
    if not isinstance(nuclei_labels, da.Array):
        nuclei_labels = da.from_array(nuclei_labels, chunks=chunk_size)

    # Function to process each chunk
    def process_chunk(chunk, dilation_radius, erosion_radius):
        # Dilate nuclei labels
        dilated = cle.dilate_labels(chunk, radius=dilation_radius)
        dilated = cle.pull(dilated)

        # Optionally erode nuclei labels
        if erosion_radius > 0:
            eroded = cle.erode_labels(chunk, radius=erosion_radius)
            eroded = cle.pull(eroded)
            nuclei = eroded
        else:
            nuclei = chunk

        # Subtract the nuclei from the dilated regions
        cytoplasm = dilated.copy()
        unique_labels = np.unique(nuclei)
        for label in unique_labels:
            if label != 0:  # Ignore background
                cytoplasm[nuclei == label] = 0
        return cytoplasm

    # Map the processing function to each chunk
    cytoplasm = nuclei_labels.map_blocks(
        process_chunk,
        dtype=nuclei_labels.dtype,
        dilation_radius=dilation_radius,
        erosion_radius=erosion_radius,
    )

    return cytoplasm


In [5]:
cytoplasm = simulate_cytoplasm_dask(nuclei_labels, dilation_radius=2, erosion_radius=0, chunk_size=(512, 512))

In [6]:
from utils_cellpose import simulate_cytoplasm

In [7]:
cytoplasm_og = simulate_cytoplasm(nuclei_labels, dilation_radius = 2, erosion_radius = 0)

c:\Users\adiezsanchez\miniforge3\envs\microglia_GPU\lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


In [8]:
import napari

In [9]:
viewer = napari.Viewer(ndisplay=2)

viewer.add_labels(nuclei_labels)
viewer.add_labels(cytoplasm_og)
viewer.add_labels(cytoplasm)

Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


<Labels layer 'cytoplasm' at 0x1f940ea4250>

In [11]:
cytoplasm

dask.array<process_chunk, shape=(3803, 2891), dtype=uint16, chunksize=(512, 512), chunktype=numpy.ndarray>

In [12]:
cytoplasm_og

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint32)

In [13]:
cytoplasm_ero = simulate_cytoplasm_dask(nuclei_labels, dilation_radius=2, erosion_radius=1, chunk_size=(512, 512))

c:\Users\adiezsanchez\miniforge3\envs\microglia_GPU\lib\site-packages\pyopencl\cache.py:417: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  prg.build(options_bytes, [devices[i] for i in to_be_built_indices])
c:\Users\adiezsanchez\miniforge3\envs\microglia_GPU\lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


In [ ]:
import dask.array as da
import numpy as np
import pyclesperanto_prototype as cle
from dask.diagnostics import ProgressBar
from dask.distributed import Client

def simulate_cytoplasm_dask_with_progress(nuclei_labels, dilation_radius=2, erosion_radius=0, chunk_size=(512, 512)):
    """
    Simulate the cytoplasm from nuclei labels using Dask for large datasets and monitor progress.

    Args:
        nuclei_labels (ndarray or dask.array): Input nuclei labels array.
        dilation_radius (int): Radius for dilating nuclei.
        erosion_radius (int): Radius for eroding nuclei.
        chunk_size (tuple): Chunk size for Dask array.

    Returns:
        ndarray: Cytoplasm simulation as a NumPy array (computed).
    """
    # Convert nuclei_labels to a Dask array if not already
    if not isinstance(nuclei_labels, da.Array):
        nuclei_labels = da.from_array(nuclei_labels, chunks=chunk_size)

    # Function to process each chunk
    def process_chunk(chunk, dilation_radius, erosion_radius):
        # Dilate nuclei labels
        dilated = cle.dilate_labels(chunk, radius=dilation_radius)
        dilated = cle.pull(dilated)

        # Optionally erode nuclei labels
        if erosion_radius > 0:
            eroded = cle.erode_labels(chunk, radius=erosion_radius)
            eroded = cle.pull(eroded)
            nuclei = eroded
        else:
            nuclei = chunk

        # Subtract the nuclei from the dilated regions
        cytoplasm = dilated.copy()
        unique_labels = np.unique(nuclei)
        for label in unique_labels:
            if label != 0:  # Ignore background
                cytoplasm[nuclei == label] = 0
        return cytoplasm

    # Map the processing function to each chunk
    cytoplasm = nuclei_labels.map_blocks(
        process_chunk,
        dtype=nuclei_labels.dtype,
        dilation_radius=dilation_radius,
        erosion_radius=erosion_radius,
    )

    # Start the Dask client
    with Client() as client:
        print(f"Dask dashboard running at: {client.dashboard_link}")

        # Use the progress bar to monitor computation
        with ProgressBar():
            result = cytoplasm.compute()  # Trigger computation

    return result
